<a href="https://colab.research.google.com/github/aliu-7/Molecular-Property-Prediction-and-Optimization/blob/main/4_2_3_Constructing_Molecular_Graphs_with_PyTorch_Geometric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation (Google Colab)

In [ ]:
# PyTorch and PyTorch Geometric setup (Colab)
!pip install -q rdkit
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
!pip install -q torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.0/494.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.5 MB/s eta 0:00:00


# Example: Convert a Single SMILES to a PyG Graph

In [ ]:
from rdkit import Chem
from rdkit.Chem import rdmolops
import torch
from torch_geometric.data import Data

# Helper function to get atom features
def atom_features(atom):
    return torch.tensor([
        atom.GetAtomicNum(),
        atom.GetDegree(),
        int(atom.GetIsAromatic())
    ], dtype=torch.float)

# Helper function to get bond features
def bond_features(bond):
    bond_type = bond.GetBondType()
    return torch.tensor([
        int(bond_type == Chem.rdchem.BondType.SINGLE),
        int(bond_type == Chem.rdchem.BondType.DOUBLE),
        int(bond_type == Chem.rdchem.BondType.TRIPLE),
        int(bond_type == Chem.rdchem.BondType.AROMATIC),
        int(bond.GetIsConjugated())
    ], dtype=torch.float)

# Convert SMILES to molecular graph
def smiles_to_pyg(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    # Node features
    atom_feats = [atom_features(atom) for atom in mol.GetAtoms()]
    x = torch.stack(atom_feats)  # Shape: [num_nodes, num_node_features]

    # Edge list and edge features
    edge_index = []
    edge_attr = []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edge_index += [[i, j], [j, i]]  # undirected edges
        edge_feat = bond_features(bond)
        edge_attr += [edge_feat, edge_feat]

    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    edge_attr = torch.stack(edge_attr)

    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

# Test with a molecule
graph = smiles_to_pyg("CCO")  # Ethanol
print(graph)


/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "


Data(x=[3, 3], edge_index=[2, 4], edge_attr=[4, 5])


# Practice Problem 1: Visualizing Molecular Graph Features
**Task:**

- Use RDKit to parse a molecule of your choice.
- Extract and print atom and bond features using the smiles_to_pyg() function.
- Try SMILES like: "c1ccccc1O" (phenol) or "CC(=O)O" (acetic acid)

In [ ]:
# Practice Problem 1: Visualizing features for acetic acid
your_smiles = "CC(=O)O"  # Acetic acid

graph = smiles_to_pyg(your_smiles)

print("Node Features (x):")
print(graph.x)

print("\nEdge Index (edge_index):")
print(graph.edge_index)

print("\nEdge Features (edge_attr):")
print(graph.edge_attr)


Node Features (x):
tensor([[6., 1., 0.],
        [6., 3., 0.],
        [8., 1., 0.],
        [8., 1., 0.]])

Edge Index (edge_index):
tensor([[0, 1, 1, 2, 1, 3],
        [1, 0, 2, 1, 3, 1]])

Edge Features (edge_attr):
tensor([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 1.],
        [0., 1., 0., 0., 1.],
        [1., 0., 0., 0., 1.],
        [1., 0., 0., 0., 1.]])
